In [24]:
import streamlit as st
import pandas as pd
import base64
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import os

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

st.title('NFL Football Stats and Predictor')

st.markdown("""
This app provides NFL Football stats data!
*  We will also give a prediction on the playoff games for this year!
""")

#st.sidebar.header('Playoff Teams')
#selected_year = st.sidebar.selectbox('Year', list(reversed(range(2015,2021))))
st.sidebar.header('Playoff Teams')

DeltaGenerator(_root_container=1, _provided_cursor=None, _parent=DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None), _block_type=None, _form_data=None)

In [25]:
# Sidebar - Team selection
sorted_unique_team = ["cin","oti","buf","kan","gnb","tam","dal","ram"]
selected_team = st.sidebar.selectbox("Teams", sorted_unique_team)

# Web scraping of NFL player stats
# "https://www.pro-football-reference.com/teams/ram/2021.htm"   
# "https://www.pro-football-reference.com/teams/" + str(team) + "/2021.htm"
#@st.cache
def load_data(team): #year,
    url = "https://www.pro-football-reference.com/teams/" + str(selected_team) + "/2021.htm"
    df = pd.read_html(url, header = 1)
    df = df[1]
    return df
teamstats = load_data(selected_team) #selected_year, 

st.subheader('Display Selected Team Schedule & Game Results')
st.dataframe(teamstats)

# https://discuss.streamlit.io/t/how-to-download-file-in-streamlit/1806
def filedownload(df):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()  # strings <-> bytes conversions
    href = f'<a href="data:file/csv;base64,{b64}" download="playerstats.csv">Download CSV File</a>'
    return href

st.markdown(filedownload(teamstats), unsafe_allow_html=True)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [27]:
path = ("./resources/Team_df.csv")
com_data = pd.read_csv(path)
print(com_data.shape)


(224, 21)


In [28]:
# get index of every team's data
team_index = com_data['Team']

# Remove Opponent, Score, Result
mlp_model_data = com_data[['Team', 'Opp', 'TmScore', 'O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO', 'Home']]
#  , 'ADA_Pred_Mean', 'LOG_Prediction'

# change to season stats
season_stats = ['O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO']

mlp_model_data[season_stats] = mlp_model_data[season_stats] * 16

# standardise the data
from sklearn import preprocessing

sd_data = ['O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO']
#  , 'ADA_Pred_Mean', 'LOG_Prediction'

mlp_model_data[sd_data] = preprocessing.scale(mlp_model_data[sd_data])

#get indexs of every teams
team_index = com_data['Team']

mlp_model_data = pd.get_dummies(mlp_model_data)


2022-01-11 17:12:40.497 NumExpr defaulting to 8 threads.
/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [34]:
model = pickle.load(open('MLP_model.sav','rb'))

In [35]:
#model = open('MLP_model.pkl', 'rb')
#classifier = pickle.load(model)

In [36]:
pd.options.display.max_rows = None

pd.options.display.max_columns = None

# Create playoff test dataset from season averages

def Score_Predictor(home_team, away_team):
    team1 = home_team
    team2 = away_team
    
    team1_data = mlp_model_data[com_data['Team'] == team1].drop('TmScore', axis=1).reset_index(drop=True)
    team2_data = mlp_model_data[com_data['Team'] == team2].drop('TmScore', axis=1).reset_index(drop=True)
    
    week_slice = slice(0,16)
    
    #1 Remove if no team names
    team1_test = pd.DataFrame(team1_data[week_slice].mean(axis=0)).T #select week to use as average
    team1_test
    opp_columns = team1_test.filter(like='Opp').columns
    
    team1_test[opp_columns] = 0
    team1_test['Opp_' + team2] = 1
    team1_test['Home'] = 1
    
    #2
    team2_test = pd.DataFrame(team2_data[week_slice].mean(axis=0)).T #select week to use as average
    opp_columns = team2_test.filter(like='Opp').columns
    
    team2_test[opp_columns] = 0
    team2_test['Opp_' + team1] = 1
    team2_test['Home'] = 1 # change to remove home field advantage
    
    # head to head matchup
    team1_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team2_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    team2_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team1_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    
    X_Playoff_test = pd.concat([team1_test, team2_test])
    X_Playoff_test.fillna(0, inplace = True) # added to address the NANs that was causing the error
    
    scores = model.predict(X_Playoff_test)
    print(team1, "will score", round(scores[0], 1))
    print(team2, "will score", round(scores[1], 1))
    
    if scores[0] > scores[1]:
        winner = team1
    else:
        winner = team2
        
    print(winner, "are the WINNERS!!!")
    
    return scores, winner

In [37]:
scores, winner = Score_Predictor('Philadelphia Eagles', 'Tampa Bay Buccaneers')

Philadelphia Eagles will score 26.2
Tampa Bay Buccaneers will score 29.2
Tampa Bay Buccaneers are the WINNERS!!!


In [14]:
def predict_team(Buffalo Bills, Pittsburgh Steelers, Kansas City Chiefs,Las Vegas Raiders,
                      Tennessee Titans,Los Angeles Rams,New England Patriots,Tampa Bay Buccaneers,
                      San Fransisco 49ers,Cincinnati Bengals,Dallas Cowboys,Philadelphia Eagles,
                      Green Bay Packers,Arizona Cardinals):
    input=np.array([[Length,Diameter,Height,Whole_weight,Shucked_weight,
                     Viscera_weight,Shell_weight]]).astype(np.float64)
    prediction = model.predict(input)
    
    return int(prediction)

SyntaxError: invalid syntax (1478235338.py, line 1)

In [7]:
col1, col2 = st.columns(2)
col1.header("teams")

col2.header("Prediction")

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [ ]:
button1 = st.button("Run Prediction")
if button1:
    st.write("prediction are being calculated...")
    result = predict_model()
    
    


In [ ]:
#test

In [10]:
bills = st.image("chiefslogo.png", width=40)

In [ ]:
# @st.cache(allow_output_mutation=True)
def load_model(model_name):
    LR = LR.load(model_name)
    return (LR)
LR = load_model('model_name')

In [ ]:
def load_model():
    model = tf.keras.models.load_model("./resourses/model.hdf5")
    return model
model = load_model()

In [6]:
st.sidebar.image("chiefslogo.png", width=40)

DeltaGenerator(_root_container=1, _provided_cursor=None, _parent=DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None), _block_type=None, _form_data=None)

In [11]:
# saving the model to a file to be able to use it later in streamlit. save it in same folder as the .py file
import pickle
pickle.dump(lr, open(' ./model.sav', 'wb'))

In [ ]:
#load model in streamlit
model = pickle.load(open('model.sav', 'rb'))

In [ ]:
# Fit the model on training set
model = LogisticRegression()
model.fit(X_train, Y_train)
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)    
 
